# Import Data and Clean

In [4]:
import pandas as pd 
from backtesting import Strategy
from backtesting.lib import crossover

# Covert Date to Date and times
df = pd.read_csv("FUND_US_XNAS_VSMAX_9_8_2024_9_8_2025.csv", parse_dates=['Date'], index_col='Date')

# Remove possible timezone data
df.index = df.index.tz_localize(None)

print(df.head())

              Open    High     Low   Close
Date                                      
2025-09-05  122.47  122.47  122.47  122.47
2025-09-04  121.61  121.61  121.61  121.61
2025-09-03  120.15  120.15  120.15  120.15
2025-09-02  120.33  120.33  120.33  120.33
2025-08-29  121.03  121.03  121.03  121.03


# Simple Moving Average

In [30]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [31]:
from backtesting import Strategy
from backtesting.lib import crossover

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [32]:
import pandas as pd
from backtesting.lib import SignalStrategy, TrailingStrategy


class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

In [36]:
from backtesting import Backtest
from backtesting.test import GOOG

bt = Backtest(df, SmaCross, commission=.002)

stats = bt.run()


# Things we care about -> Sharpe Ratio , Return [%] , Buy & Hold Return [%] , Max. Drawdown [%]
# Avg. Drawdown [%]                     
# Max. Drawdown Duration      
# Avg. Drawdown Duration       

# Most important stats, currenty this strategy sucks
def retrieve(stats):
    print("STRATEGY -> ",stats["_strategy"])
    print("Sharpe ratio -> ", stats["Sharpe Ratio"])
    print("Return % -> ", stats["Return [%]"])
    print("Buy and Hold Return -> ", stats["Buy & Hold Return [%]"])
    print("Max. Drawdown [%] -> ", stats["Max. Drawdown [%]"])
    print("Avg. Drawdown [%]  -> ", stats["Avg. Drawdown [%]"])
    print("Max. Drawdown Duration -> ", stats["Max. Drawdown Duration"])
    print("Avg. Drawdown Duration  -> ", stats["Avg. Drawdown Duration"])

C:\Users\zuria\AppData\Local\Temp\ipykernel_16064\984125041.py:4: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(df, SmaCross, commission=.002)
C:\Users\zuria\AppData\Local\Temp\ipykernel_16064\984125041.py:6: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = bt.run()


In [37]:
%%time

stats = bt.optimize(n1=range(1,10),
                    n2=range(1,10),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
retrieve(stats)

c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()
c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=13: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=16: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
c:\Users\zu

STRATEGY ->  SmaCross(n1=2,n2=6)
Sharpe ratio ->  1.3687345618383502
Return % ->  22.377323799999996
Buy and Hold Return ->  13.272290048094703
Max. Drawdown [%] ->  -8.327947341813458
Avg. Drawdown [%]  ->  -1.9137432272750479
Max. Drawdown Duration ->  158 days 00:00:00
Avg. Drawdown Duration  ->  22 days 00:00:00
CPU times: total: 1.69 s
Wall time: 1.8 s


c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=27: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=200: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=202: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=222: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
c:\Users\zuria\Projects\backtesting-VSMAX\.venv\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest